In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import sys
print (sys.version)

2.7.12 |Anaconda 4.2.0 (64-bit)| (default, Jun 29 2016, 11:07:13) [MSC v.1500 64 bit (AMD64)]


In [2]:
import pandas as pd #CONNECT TO THE PANDAS LIBRARY
import numpy as np #IMPORT NUMPY

In [3]:
import datetime
now = datetime.datetime.now()
#day="%s/%s" % (now.month-1,now.year)
#day_save="%s_%s" % (now.month-1,now.year)
#year=now.year
#month=now.month

#day=day
#print day

#day_save=day_save
#print day_save

#month=month-1
#print month
month=4
day=201804

In [4]:
acq_col_names=['STATUSMONTH',
           'STATUSYEAR',
           'ORDERID',
           'CONTRACT_ID',
           'CLIENT_NAME',
           'REQUESTAUTHORIZEDAMOUNT',
           'CURRENCYCODE',
           'CURRENT_PAYMENTSTATUSID_UPDATED',
           'PAYMENTPRODUCTID',
           'PAYMENTMETHODID',
           'PAYMENTPROCESSORNUMBER',
           'CREDITCARDCOMPANY',
           'COUNTRYCODE',
           'BIN',
           'INVOICE_CURRENCY',
           'SERVICEPROVIDERGROUPNAME',
           'ICPLUS_TYPE',
           'GEOGRAPHICAL_AREA_LOCAL_ACQUIRER_DESC']

In [5]:
curr_col_names=['us_conversion','eu_conversion','CURRENCYCODE','STATUSYEAR','STATUSMONTH']

In [6]:
curr=pd.read_csv('G:\\Pricing\\Pricing Intelligence\\Projects\\IC_Opt_project\\pi_implementation\\currency_conversion\\monthly\\currency_monthly_data.csv')
excep=pd.read_csv('G:\\Pricing\\Pricing Intelligence\\Projects\\IC_Opt_project\\pi_implementation\\misuse_of_authorization_fee\\reference_tables\\mu_exceptions_tbl.csv')

prd_ref_tbl=pd.read_csv('G:\\Pricing\\Pricing Intelligence\\Projects\\IC_Opt_project\\pi_implementation\\misuse_of_authorization_fee\\reference_tables\\vi_product_ref_tbl.csv')
prd_ref_tbl=prd_ref_tbl[['CREDITDEBITINDICATOR','CURRENCYCODE','PRODUCT_CODE']]

vi=pd.read_csv('d:\\djohnson\\Desktop\\misuse\\2018\\apr\\data_files\\visa_misuse.csv',sep=';',names=acq_col_names,skiprows=66)
con =pd.read_csv('d:\\djohnson\\Desktop\\misuse\\2018\\apr\\currency_conversion\\monthly_currency_average.csv',sep=';',names=curr_col_names,skiprows=9)

C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
vi_mu_working_data=vi #Create a working reference of the misuse of information

In [8]:
vi_mu_working_data.rename(columns={'GEOGRAPHICAL_AREA_LOCAL_ACQUIRER_DESC,': 'GEOGRAPHICAL_AREA_LOCAL_ACQUIRER_DESC'}, inplace=True)#Update the name to USD conversion factor
vi_mu_working_data['CREDITDEBITINDICATOR']='C'

In [9]:
vi['CURRENCYCODE']=vi['CURRENCYCODE'].str.strip()
vi['ICPLUS_TYPE']=vi['ICPLUS_TYPE'].str.strip()
con['CURRENCYCODE']=con['CURRENCYCODE'].str.strip()

In [10]:
con['STATUSMONTH']=4

In [11]:
vi_mu_working_data.head()

,STATUSMONTH,STATUSYEAR,ORDERID,CONTRACT_ID,CLIENT_NAME,REQUESTAUTHORIZEDAMOUNT,CURRENCYCODE,CURRENT_PAYMENTSTATUSID_UPDATED,PAYMENTPRODUCTID,PAYMENTMETHODID,PAYMENTPROCESSORNUMBER,CREDITCARDCOMPANY,COUNTRYCODE,BIN,INVOICE_CURRENCY,SERVICEPROVIDERGROUPNAME,ICPLUS_TYPE,GEOGRAPHICAL_AREA_LOCAL_ACQUIRER_DESC,CREDITDEBITINDICATOR
0,4,2018.0,3.766476e+06,6702.0,WCB - Laithwaites Wine,100.0,USD,600.0,1.0,1.0,283.0,VISA,US,422695.0,USD,Vantiv,Cost Plus,NorthAm,C
1,4,2018.0,3.786056e+06,7706.0,WSJwine - Wine Cellar-Redbank,100.0,USD,600.0,1.0,1.0,283.0,VISA,US,414720.0,USD,Vantiv,Cost Plus,NorthAm,C
2,4,2018.0,2.452671e+08,4235.0,Valve Cards US,100.0,USD,600.0,1.0,1.0,372.0,VISA,US,445178.0,USD,Vantiv,Cost Plus,NorthAm,C
3,4,2018.0,1.233219e+09,6601.0,Tourico Holidays (B2B - US BIN,200.0,USD,600.0,1.0,1.0,283.0,VISA,US,414720.0,USD,Vantiv,Cost Plus,NorthAm,C
4,4,2018.0,1.028886e+08,3911.0,Linden Lab,100.0,USD,600.0,1.0,1.0,283.0,VISA,US,446561.0,USD,Vantiv,-,NorthAm,C


In [12]:
vi_mu_working_data['PROCESS_INTEGRITY_FEE']=1*0.09

In [13]:
vi_mu_working_data['REQUESTAUTHORIZEDAMOUNT']=vi_mu_working_data['REQUESTAUTHORIZEDAMOUNT']/100

In [14]:
vi_mu_working_data=vi_mu_working_data[['STATUSMONTH','STATUSYEAR','CLIENT_NAME','CONTRACT_ID','SERVICEPROVIDERGROUPNAME','GEOGRAPHICAL_AREA_LOCAL_ACQUIRER_DESC','ORDERID','CREDITDEBITINDICATOR','CURRENCYCODE','REQUESTAUTHORIZEDAMOUNT','INVOICE_CURRENCY','ICPLUS_TYPE','PROCESS_INTEGRITY_FEE']]

In [15]:
vi_mu_working_data=pd.merge(vi_mu_working_data,con,on=['CURRENCYCODE','STATUSMONTH','STATUSYEAR'])
vi_mu_working_data['REQUESTAUTHORIZEDAMOUNT']=vi_mu_working_data['REQUESTAUTHORIZEDAMOUNT']/vi_mu_working_data['us_conversion']

In [16]:
vi_mu_working_data=pd.merge(vi_mu_working_data,prd_ref_tbl,on=['CURRENCYCODE','CREDITDEBITINDICATOR'],how='left')

In [17]:
vi_mu_working_data=pd.merge(vi_mu_working_data,excep,on=['CONTRACT_ID'],how='left')

In [18]:
vi_mu_working_data['ICPLUS_TYPE'][(vi_mu_working_data['CONTRACT_ID']==7162)]="-"
vi_mu_working_data['INVOICE_CURRENCY'][(vi_mu_working_data['CONTRACT_ID']==7162)]="USD"

C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [19]:
vi_mu_working_data=pd.merge(vi_mu_working_data,curr,on=['CURRENCYCODE'],how='left')
vi_mu_working_data.rename(columns={'EUR_CONVERSION_RATE': 'USD_CONVERSION_FACTOR'}, inplace=True)#Update the name to USD conversion factor

In [20]:
curr_invoice=curr
curr_invoice.rename(columns={'CURRENCYCODE': 'INVOICE_CURRENCY'}, inplace=True)

In [21]:
vi_mu_working_data=pd.merge(vi_mu_working_data,curr_invoice,on=['INVOICE_CURRENCY'],how='left')
vi_mu_working_data['INVOICE_CONVERSION_FACTOR']=vi_mu_working_data['EUR_CONVERSION_RATE']

In [22]:
vi_mu_working_data['CURRENCY_ADJUSTED_INVOICE_AMOUNT']=vi_mu_working_data['PROCESS_INTEGRITY_FEE']/vi_mu_working_data['INVOICE_CONVERSION_FACTOR']

In [23]:
vi_mu_working_data['INVOICE_CURRENCY'][(vi_mu_working_data['INVOICE_CURRENCY'].isnull())&(vi_mu_working_data['CURRENCYCODE']=='CAD')]='USD'

C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [24]:
curr[(curr['INVOICE_CURRENCY']=='EUR')]

usd=curr[:].iloc[4]['EUR_CONVERSION_RATE']#Establish the European Conversion Amount
vi_mu_working_data['EUR_TOTAL']=0
vi_mu_working_data['EUR_TOTAL'][(vi_mu_working_data['INVOICE_CURRENCY']!="EUR")]=vi_mu_working_data['CURRENCY_ADJUSTED_INVOICE_AMOUNT']/usd
vi_mu_working_data['EUR_TOTAL'][(vi_mu_working_data['INVOICE_CURRENCY']=="EUR")]=vi_mu_working_data['CURRENCY_ADJUSTED_INVOICE_AMOUNT']

C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [25]:
vi_mu_working_data['Product Description']='Penalty Fees'

vi_mu_working_data['Current % Tariff']=0
vi_mu_working_data['Current % Tariff'][(vi_mu_working_data['EXCEPTION']=='YES')]=1
vi_mu_working_data['Current % Tariff'][(vi_mu_working_data['EXCEPTION']!='YES')]=1.10

vi_mu_working_data['Description']=''
vi_mu_working_data['Description'][(vi_mu_working_data['EXCEPTION']=='YES')]='Fee 100.00% or as currently in IST ='
vi_mu_working_data['Description'][(vi_mu_working_data['EXCEPTION']!='YES')]='Fee 110.00% or as currently in IST ='

vi_mu_working_data['Product Code']=420129

C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipy

In [26]:
vi_mu_working_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33704 entries, 0 to 33703
Data columns (total 28 columns):
STATUSMONTH                              33704 non-null object
STATUSYEAR                               33704 non-null float64
CLIENT_NAME                              33704 non-null object
CONTRACT_ID                              33704 non-null float64
SERVICEPROVIDERGROUPNAME                 33704 non-null object
GEOGRAPHICAL_AREA_LOCAL_ACQUIRER_DESC    33704 non-null object
ORDERID                                  33704 non-null float64
CREDITDEBITINDICATOR                     33704 non-null object
CURRENCYCODE                             33704 non-null object
REQUESTAUTHORIZEDAMOUNT                  33704 non-null float64
INVOICE_CURRENCY                         33704 non-null object
ICPLUS_TYPE                              33704 non-null object
PROCESS_INTEGRITY_FEE                    33704 non-null float64
us_conversion                            33704 non-null float64
eu_

In [27]:
vi_mu_working_data['Total Euros Collected (including current tariff)']=vi_mu_working_data['EUR_TOTAL']*vi_mu_working_data['Current % Tariff']
vi_mu_working_data['Invoice Amount (including tariff)']=vi_mu_working_data['CURRENCY_ADJUSTED_INVOICE_AMOUNT']*vi_mu_working_data['Current % Tariff']

In [28]:
vi_mu_working_data_rev=vi_mu_working_data[['STATUSMONTH'
                          ,'STATUSYEAR'
                          ,'CLIENT_NAME'
                          ,'CONTRACT_ID'
                          ,'SERVICEPROVIDERGROUPNAME'
                          ,'PRODUCT_CODE'
                          ,'GEOGRAPHICAL_AREA_LOCAL_ACQUIRER_DESC'
                          ,'ORDERID'
                          ,'CREDITDEBITINDICATOR'
                          ,'CURRENCYCODE'
                          ,'INVOICE_CURRENCY'                         
                          ,'REQUESTAUTHORIZEDAMOUNT'
                          ,'PROCESS_INTEGRITY_FEE'                                         
                          ,'INVOICE_CONVERSION_FACTOR'
                          ,'CURRENCY_ADJUSTED_INVOICE_AMOUNT'
                          ,'EUR_TOTAL'
                          ,'EXCEPTION'                         
                          ,'ICPLUS_TYPE']]

In [29]:
vi_mu_working_data_pivot=vi_mu_working_data_rev

In [30]:
vi_mu_working_data_pivot['EUR_TOTAL'][(vi_mu_working_data_pivot['EXCEPTION']=='YES')]=vi_mu_working_data_pivot['EUR_TOTAL']*1
vi_mu_working_data_pivot['EUR_TOTAL'][(vi_mu_working_data_pivot['EXCEPTION']!='YES')]=vi_mu_working_data_pivot['EUR_TOTAL']*1.10

C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\pandas\core\generic.py:4485: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versu

In [31]:
vi_mu_working_data_pivot=vi_mu_working_data_pivot[(vi_mu_working_data_pivot['STATUSMONTH']>=1)]
print vi_mu_working_data_pivot.pivot_table(['EUR_TOTAL'],index=['STATUSMONTH'],columns='ICPLUS_TYPE',aggfunc='sum',margins='all')

              EUR_TOTAL                                     
ICPLUS_TYPE           -    Cost Plus IC+SF Plus          All
STATUSMONTH                                                 
4            658.914545  1778.228182     576.36  3013.502727
All          658.914545  1778.228182     576.36  3013.502727


In [32]:
vi_mu_working_data_pivot=vi_mu_working_data_pivot[(vi_mu_working_data_pivot['STATUSMONTH']>=1)]
print vi_mu_working_data_pivot.pivot_table(['ORDERID'],index=['STATUSMONTH'],columns='ICPLUS_TYPE',aggfunc='count',margins='all')

            ORDERID                              
ICPLUS_TYPE       - Cost Plus IC+SF Plus      All
STATUSMONTH                                      
4            7484.0   19816.0     6404.0  33704.0
All          7484.0   19816.0     6404.0  33704.0


In [33]:
vi_mu_working_data_rev_rf=vi_mu_working_data_rev
vi_mu_working_data_rev_rf['Product Description']='Penalty Fees'

vi_mu_working_data_rev_rf['Current % Tariff']=0
vi_mu_working_data_rev_rf['Current % Tariff'][(vi_mu_working_data_rev_rf['EXCEPTION']=='YES')]=1
vi_mu_working_data_rev_rf['Current % Tariff'][(vi_mu_working_data_rev_rf['EXCEPTION']!='YES')]=1.10

vi_mu_working_data_rev_rf['Description']=''
vi_mu_working_data_rev_rf['Description'][(vi_mu_working_data_rev_rf['EXCEPTION']=='YES')]='Fee 100.00% or as currently in IST ='
vi_mu_working_data_rev_rf['Description'][(vi_mu_working_data_rev_rf['EXCEPTION']!='YES')]='Fee 110.00% or as currently in IST ='



vi_mu_working_data_rev_rf['Product Code']=420129
vi_mu_working_data_rev_rf['Total invoiced amount']=0

vi_mu_working_data_rev_rf=vi_mu_working_data_rev_rf[(vi_mu_working_data_rev_rf['STATUSMONTH']==month)&(vi_mu_working_data_rev_rf['ICPLUS_TYPE']=='-')]
vi_mu_working_data_rev_rf=vi_mu_working_data_rev_rf[['CONTRACT_ID'
                             ,'Product Code'
                             ,'Total invoiced amount'                            
                             ,'Product Description'
                             ,'INVOICE_CURRENCY'
                             ,'ORDERID'
                             ,'REQUESTAUTHORIZEDAMOUNT'
                             ,'Current % Tariff'
                             ,'CURRENCY_ADJUSTED_INVOICE_AMOUNT'
                             ,'Description']]



C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: ht

In [34]:
vi_mu_working_data_rev_rf=vi_mu_working_data_rev_rf.groupby(['CONTRACT_ID','Product Code','Product Description','INVOICE_CURRENCY','Current % Tariff','Description','Total invoiced amount'])\
.agg({'ORDERID':'count','REQUESTAUTHORIZEDAMOUNT':'sum','CURRENCY_ADJUSTED_INVOICE_AMOUNT':'sum'}).reset_index()

vi_mu_working_data_rev_rf['Current Tariff Amount']=vi_mu_working_data_rev_rf['CURRENCY_ADJUSTED_INVOICE_AMOUNT']/vi_mu_working_data_rev_rf['ORDERID']
vi_mu_working_data_rev_rf['Total invoiced percentage']=vi_mu_working_data_rev_rf['CURRENCY_ADJUSTED_INVOICE_AMOUNT']*vi_mu_working_data_rev_rf['Current % Tariff']

vi_mu_working_data_rev_rf.rename(columns={'CONTRACT_ID': 'Contract ID'
                                            ,'Product Code': 'Product Code'
                                            ,'INVOICE_CURRENCY': 'Invoice Currency'
                                            ,'ORDERID':'Volume'
                                            ,'CURRENCY_ADJUSTED_INVOICE_AMOUNT':'Flow'}, inplace=True)


vi_mu_working_data_rev_rf=vi_mu_working_data_rev_rf[['Contract ID'
                             ,'Product Code'
                             ,'Product Description'
                             ,'Invoice Currency'
                             ,'Volume'
                             ,'Flow'
                             ,'Current % Tariff'
                             ,'Current Tariff Amount'
                             ,'Total invoiced amount'                            
                             ,'Total invoiced percentage'
                             ,'Description']]


vi_mu_working_data_rev_rf['Volume']=0
vi_mu_working_data_rev_rf['Current Tariff Amount']=0

In [35]:
#display the invoice data
vi_mu_working_data_rev_rf['Approved']='FALSE'
vi_mu_working_data_rev_rf['InvoiceDate']=day
vi_mu_working_data_rev_rf


vi_mu_working_data_rev_rf_final=vi_mu_working_data_rev_rf[['Contract ID','Product Code','Total invoiced percentage','InvoiceDate','Approved']]
vi_mu_working_data_rev_rf_final.rename(columns={'Contract ID': 'Contract_ID'
                                            ,'Product Code':'ProductCode'
                                            ,'Total invoiced percentage': 'Value'}, inplace=True)

vi_mu_working_data_rev_rf_final['Contract_ID']=vi_mu_working_data_rev_rf_final['Contract_ID'].astype(str)
vi_mu_working_data_rev_rf_final['Contract_ID'][(vi_mu_working_data_rev_rf_final['Contract_ID'].str.len()==3)]="0"+vi_mu_working_data_rev_rf_final['Contract_ID']

#vi_mu_working_data_rev_rf_final=vi_mu_working_data_rev_rf_final.set_index('Contract_ID')

print vi_mu_working_data_rev_rf_final.head()

# ## Export the Data for Use and Analysis

#vi_mu_working_data_rev_rf.to_csv('G:\\Pricing\\Pricing Intelligence\\Misuse of Authorization\\mtm\\vi_misuse_month_add_rev_'+day_save+'.csv')
#mc_mu_working_data_rev_rf_final.to_csv('d:\\djohnson\\Pricing Desktop\\misuse\\vi_integ_fee_ytd'+day_save+'.csv',sep=';')
vi_mu_working_data_rev_rf_final.to_csv('d:\\djohnson\\Desktop\\misuse\\2018\\apr\\invoice_files\\vi_integ.csv',sep=';')
vi_mu_working_data.to_csv('d:\\djohnson\\Desktop\\misuse\\2018\\apr\\audit_files\\vi_integ_fee_ytd.csv')

C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\pandas\core\frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)
C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


  Contract_ID  ProductCode   Value  InvoiceDate Approved
0       764.0       420129   0.090       201804    FALSE
1       766.0       420129   0.270       201804    FALSE
2       991.0       420129   9.603       201804    FALSE
3      1086.0       420129  48.906       201804    FALSE
4      2415.0       420129   1.485       201804    FALSE
